In [53]:
import os
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.io import imread
import skimage
import ipywidgets as widgets
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, Normalize

# %matplotlib notebook
%matplotlib ipympl

In [57]:
dataset_root = "/globalscratch/users/n/s/nsayez/deepsun_bioblue/All"
# seg_types = ['LTH_UnsupervisedLonePenumbra']
seg_types = ['LTH_LonePenumbraIsBg']

image_folder     = os.path.join(dataset_root, 'image')
# gt_folder        = os.path.join(dataset_root, 'GroundTruth')
generated_folders = {t : os.path.join(dataset_root, t) for t in seg_types}

image_lst = sorted(glob.glob(os.path.join(image_folder, '**/*.FTS'), recursive=True))

generated_lsts = {t : sorted(glob.glob(os.path.join(generated_folders[t], '*.png'))) for t in seg_types}

print(len(image_lst))
# print(len(generated_lsts['LTH_UnsupervisedLonePenumbra']))
print(len(generated_lsts['LTH_LonePenumbraIsBg']))

5991
5991


In [56]:
for t in seg_types:
    
    for img in generated_lsts[t]:
        if os.path.basename(img).startswith('UCC'):
            os.remove(img)

# print(len(generated_lsts['LTH_LonePenumbraIsBg']))

In [58]:
max_idx = len(image_lst)

cmap_gen = cm.turbo
# cmap_pred = cm.cool
cmap_gen = cmap_gen(range(255))
cmap_gen = ListedColormap([(0, 0, 0, 0), *cmap_gen])

cmap_gt = cm.autumn
cmap_gt = cmap_gt(range(255))
cmap_gt = ListedColormap([(0, 0, 0, 0), *cmap_gt])


def refresh(slider): 
    xlims0 = axes0[0].get_xlim()
    ylims0 = axes0[0].get_ylim()
    
    axes0[0].clear()
    
    test_img = imread(image_lst[idx_slider.value])
#     gt_label = imread(gt_lst[idx_slider.value])
#     gen_label = imread(generated_lst[idx_slider.value])
    
    if img_cb.value:
        axes0[0].imshow(test_img, cmap="gray", interpolation="None")
#         axes0[0].invert_yaxis()
        axes0[0].get_xlim()[::-1]
    if gt_cb.value:
        axes0[0].imshow(gt_label, cmap=cmap_gt, interpolation="None", alpha=.5)
    for i, cb in enumerate(gen_cbs):
        if cb.value:
            tmp = imread(generated_lsts[seg_types[i]][idx_slider.value])
            cs = axes0[0].imshow(tmp, cmap=cmap_gen, interpolation="None", alpha=.5)
        
    if xlims0 != (0.0, 1.0):
        axes0[0].set_xlim(xlims0)
        axes0[0].set_ylim(ylims0)
    return

max_rows = 1
max_cols = 1

plt.ioff()
plt.style.use('default')
# plt.style.use('dark_background')
fig_widget0, axes0 = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(8,5))

try:
    len(axes0)
except TypeError:
    axes0 = [axes0]

plt.ion()
img_cb = widgets.Checkbox(value=True, description='Show img')
gt_cb = widgets.Checkbox(value=False, description='Show gt')
# ann_cbs = [widgets.Checkbox(value=False, description=f'Show {exp}') for exp in experts ]
gen_cbs = [widgets.Checkbox(value=False, description=f'Show {t}') for t in seg_types]
# gen_cb = widgets.Checkbox(value=False, description='Show Generation')
idx_slider = widgets.IntSlider(value=0, min=0, max=max_idx-1, step=1, description="Image Index")


# Input image to predict
test_img = imread(image_lst[0])
#prediction
gen_label = imread(generated_lsts[seg_types[0]][0])


print(test_img.dtype)
print(gen_label.dtype)

axes0[0].imshow(test_img, cmap="gray", interpolation="None")

fig_widget0.colorbar(cm.ScalarMappable(norm=Normalize(vmin=0, vmax=3, clip=False), cmap=cmap_gen), ax=axes0)
# fig_widget0.colorbar(cs)

img_cb.observe(refresh, names='value')
gt_cb.observe(refresh, names='value')
for cb in gen_cbs:
    cb.observe(refresh, names='value')
# gen_cb.observe(refresh, names='value')
idx_slider.observe(refresh, names='value')


widgets.VBox([widgets.HBox([idx_slider, img_cb]+gen_cbs), fig_widget0.canvas])

uint16
uint8


In [41]:
print(image_lst[0])

/globalscratch/users/n/s/nsayez/deepsun_bioblue/All/image/2013/01/UPH20130101133418.FTS


In [4]:
axes0[0].invert_yaxis()
axes0[0].set_ylim([1125,1325])
axes0[0].set_xlim([630,1025])
cmap_pred = cm.turbo
# cmap_pred = cm.cool
cmap_pred = cmap_pred(range(255))
cmap_pred = ListedColormap([(0, 0, 0, 0), *cmap_pred])

In [5]:
# import torch
# import numpy as np 


# # PyTroch version

# SMOOTH = 1e-6

# def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
#     # You can comment out this line if you are passing tensors of equal shape
#     # But if you are passing output from UNet or something it will most probably
#     # be with the BATCH x 1 x H x W shape
#     outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
#     intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
#     union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
#     iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
#     thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
#     return thresholded  # Or thresholded.mean() if you are interested in average across the batch
    
    
# # Numpy version
# # Well, it's the same function, so I'm going to omit the comments

# def iou_numpy(outputs: np.array, labels: np.array):
#     outputs = outputs.squeeze(1)
    
#     intersection = (outputs & labels).sum((1, 2))
#     union = (outputs | labels).sum((1, 2))
    
#     iou = (intersection + SMOOTH) / (union + SMOOTH)
    
#     thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
#     return thresholded  # Or thresholded.mean()

In [6]:
# build tensor for the whole reference set
gt_tensors = [] 
for i, path in enumerate(gt_lst[:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    gt_tensors.append( torch.unsqueeze(torch.from_numpy(np_array), axis=0))
    
tot_gt = torch.cat(gt_tensors)
print(tot_gt.shape)

LTH_tensors = [] 
for i, path in enumerate(generated_lsts['LocalTopHat'][:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    LTH_tensors.append( torch.unsqueeze(torch.from_numpy(np_array.astype(np.uint8)), axis=0))

tot_LTH = torch.cat(LTH_tensors)
print(tot_LTH.shape)

Grid_tensors = [] 
for i, path in enumerate(generated_lsts['Grid'][:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    Grid_tensors.append( torch.unsqueeze(torch.from_numpy(np_array.astype(np.uint8)), axis=0))

tot_Grid = torch.cat(Grid_tensors)
print(tot_Grid.shape)

NameError: name 'torch' is not defined

In [148]:
from typing import Optional
import pytorch_lightning as pl
from torchmetrics import ConfusionMatrix
import torch
import tqdm


class IoU(ConfusionMatrix):
    def __init__(
        self,
        num_classes: int,
        threshold: float = 0.5,
        class_index: Optional[int] = None,
    ):
        super().__init__(
            num_classes,
            normalize=None,
            threshold=threshold,
            compute_on_step=True,
            dist_sync_on_step=False,
        )
        self.class_index = class_index

    def compute(self) -> torch.Tensor:
        cm = super().compute().to(torch.double)
        iou = cm.diag() / (cm.sum(dim=1) + cm.sum(dim=0) - cm.diag() + 1e-15)
        if self.class_index is None:
            return iou.mean()
        else:
            return iou[self.class_index]


In [149]:
LTH_ious = {'0':[], '1': [], '2':[] }
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    for c in range(3):
        iou = IoU(num_classes=3, class_index=c)
        res = iou(tot_LTH[i,:,:], tot_gt[i,:,:])
        LTH_ious[str(c)].append(res)
#         print(f'class {c} -> {res}')
# print(ious)


100%|███████████████████████████████████████████████████████| 36/36 [03:00<00:00,  5.02s/it]


In [150]:
LTH_mean_ious = {t: np.mean(LTH_ious[t]) for t in LTH_ious.keys()}
print(LTH_mean_ious)

{'0': 0.999660716157584, '1': 0.44184888613901757, '2': 0.3772279800360334}


In [151]:
# CMs are shaped [target_classes X pred_classes]
LTH_cms = []
# for i in tqdm.tqdm(range(2)):
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    LTH_cm = ConfusionMatrix(num_classes=3, normalize='true')
    LTH_cm(tot_LTH[i,:,:], tot_gt[i,:,:]).to('cuda:0')
    cm = LTH_cm.compute()
#     print(cm)
#     print((cm.sum(dim=1) + cm.sum(dim=0) - cm.diag()).sum() )
    LTH_cms.append(cm)
    
LTH_cms = [torch.unsqueeze(item, 0) for item in LTH_cms.copy()]

 50%|███████████████████████████▌                           | 18/36 [00:30<00:30,  1.69s/it]/home/ucl/elen/nsayez/miniconda3/envs/A100-bioblue/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 3 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)
100%|███████████████████████████████████████████████████████| 36/36 [01:00<00:00,  1.69s/it]


In [152]:
print(LTH_cms[1].shape)
tot_LTH_CM = torch.cat(LTH_cms)
print(tot_LTH_CM.shape)
mean_LTH_CM = torch.mean(100*tot_LTH_CM,0)
std_LTH_CM = torch.std(100*tot_LTH_CM,0)
# print(mean_LTH_CM)
# print(std_LTH_CM)

LTH_CM_percents = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(mean_LTH_CM.shape[0]):
#     print(i)
    for j in range(mean_LTH_CM.shape[1]):
#         print(j)
        LTH_CM_percents[i][j] = '{:.3f}'.format(mean_LTH_CM[i,j])
    
print(LTH_CM_percents)

torch.Size([1, 3, 3])
torch.Size([36, 3, 3])
[['99.995', '0.005', '0.000'], ['42.190', '48.376', '9.433'], ['25.666', '14.588', '56.968']]


GRID

In [153]:
# CMs are shaped [target_classes X pred_classes]
Grid_cms = []
# for i in tqdm.tqdm(range(2)):
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    Grid_cm = ConfusionMatrix(num_classes=3, normalize='true')
    Grid_cm(tot_Grid[i,:,:], tot_gt[i,:,:]).to('cuda:0')
    cm = Grid_cm.compute()
#     print(cm)
#     print((cm.sum(dim=1) + cm.sum(dim=0) - cm.diag()).sum() )
    Grid_cms.append(cm)
    
Grid_cms = [torch.unsqueeze(item, 0) for item in Grid_cms.copy()]

100%|███████████████████████████████████████████████████████| 36/36 [01:00<00:00,  1.67s/it]


In [154]:
print(Grid_cms[1].shape)
tot_Grid_CM = torch.cat(Grid_cms)
print(tot_Grid_CM.shape)
mean_Grid_CM = torch.mean(100*tot_Grid_CM,0)
std_Grid_CM = torch.std(100*tot_Grid_CM,0)
# print(mean_Grid_CM)
# print(std_Grid_CM)

Grid_CM_percents = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(mean_Grid_CM.shape[0]):
#     print(i)
    for j in range(mean_Grid_CM.shape[1]):
#         print(j)
        Grid_CM_percents[i][j] = '{:.3f}'.format(mean_Grid_CM[i,j])
    
print(Grid_CM_percents)

torch.Size([1, 3, 3])
torch.Size([36, 3, 3])
[['99.968', '0.032', '0.000'], ['24.096', '56.569', '19.335'], ['12.714', '25.996', '58.512']]


In [155]:
Grid_ious = {'0':[], '1': [], '2':[] }
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    for c in range(3):
        iou = IoU(num_classes=3, class_index=c)
        res = iou(tot_Grid[i,:,:], tot_gt[i,:,:])
        Grid_ious[str(c)].append(res)
#         print(f'class {c} -> {res}')
# print(ious)

100%|███████████████████████████████████████████████████████| 36/36 [02:59<00:00,  5.00s/it]


In [156]:
Grid_mean_ious = {t: np.mean(Grid_ious[t]) for t in Grid_ious.keys()}
print(Grid_mean_ious)

{'0': 0.9995164164985033, '1': 0.36687629560801466, '2': 0.28443676916901955}
